In [ ]:
import numpy as np
import pandas as pd
import logging

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("zernike_process_log.log"),
        logging.StreamHandler()
    ]
)

# Zernike 기저 행렬 생성 함수
def compute_zernike_matrix(r, theta, max_order=5):
    """
    Zernike 다항식 기저 생성
    :param r: 방사 좌표계의 반지름 값 (0 ≤ r ≤ 1로 스케일링 필요)
    :param theta: 방사 좌표계의 각도 값 (-π ≤ θ ≤ π)
    :param max_order: 최대 차수
    :return: Zernike 기저 행렬
    """
    Z = [np.ones_like(r)]  # Z0,0: 상수항
    for n in range(1, max_order + 1):
        for m in range(-n, n + 1, 2):
            if m >= 0:
                Z.append(r**n * np.cos(m * theta))  # 짝수 m: cos(mθ)
            else:
                Z.append(r**n * np.sin(-m * theta))  # 홀수 m: sin(-mθ)
    return np.array(Z).T  # 각 Zernike 다항식이 열로 구성된 행렬


# 좌표 변환 및 Zernike 기저 생성 함수
def prepare_zernike_coordinates(group, max_order=5):
    """
    Zernike 좌표 및 기저 행렬 생성
    :param group: UNIQUE_ID별 그룹 데이터
    :param max_order: 최대 차수
    :return: Zernike 기저 행렬
    """
    die_x = group['DieX']
    die_y = group['DieY']
    step_pitch_x = group['STEP_PITCH_X']
    step_pitch_y = group['STEP_PITCH_Y']
    map_shift_x = group['MAP_SHIFT_X']
    map_shift_y = group['MAP_SHIFT_Y']
    coordinate_x = group['coordinate_X']
    coordinate_y = group['coordinate_Y']

    # 방사 좌표계 변환
    wf_x = die_x * step_pitch_x + map_shift_x + coordinate_x
    wf_y = die_y * step_pitch_y + map_shift_y + coordinate_y
    r = np.sqrt(wf_x**2 + wf_y**2) / 1e6  # 거리 스케일링
    theta = np.arctan2(wf_y, wf_x)

    # Zernike 기저 행렬 생성
    return compute_zernike_matrix(r, theta, max_order=max_order)


# Zernike 분석 함수
def zernike_analysis(df_rawdata, max_order=5):
    """
    Zernike 회귀분석 및 잔차 계산
    :param df_rawdata: 입력 데이터
    :param max_order: Zernike 다항식 최대 차수
    :return: (df_z_coeff, df_rawdata_with_predictions)
    """
    grouped = df_rawdata.groupby('UNIQUE_ID')
    coeff_results = []

    # 원본 데이터프레임에 예측값 및 잔차 열 추가
    df_rawdata['Z_pred_x'] = np.nan
    df_rawdata['Z_pred_y'] = np.nan
    df_rawdata['Z_residual_x'] = np.nan
    df_rawdata['Z_residual_y'] = np.nan

    for unique_id, group in grouped:
        logging.info(f"Processing UNIQUE_ID: {unique_id}")

        # Zernike 기저 생성
        Z = prepare_zernike_coordinates(group, max_order=max_order)

        # 종속변수
        Y_dx = group['X_reg']
        Y_dy = group['Y_reg']

        # 회귀 계수 계산
        coeff_dx = np.linalg.lstsq(Z, Y_dx, rcond=None)[0]
        coeff_dy = np.linalg.lstsq(Z, Y_dy, rcond=None)[0]

        # 회귀 계수 저장
        coeff_result = {'UNIQUE_ID': unique_id}
        coeff_result.update({f'Z{i+1}_dx': coeff for i, coeff in enumerate(coeff_dx)})
        coeff_result.update({f'Z{i+1}_dy': coeff for i, coeff in enumerate(coeff_dy)})
        coeff_results.append(coeff_result)

        # 예측값 계산
        pred_x = Z @ coeff_dx
        pred_y = Z @ coeff_dy

        # 잔차 계산
        residual_x = Y_dx - pred_x
        residual_y = Y_dy - pred_y

        # 원본 데이터프레임에 추가
        df_rawdata.loc[group.index, 'Z_pred_x'] = pred_x
        df_rawdata.loc[group.index, 'Z_pred_y'] = pred_y
        df_rawdata.loc[group.index, 'Z_residual_x'] = residual_x
        df_rawdata.loc[group.index, 'Z_residual_y'] = residual_y

    # 회귀 계수 결과 데이터프레임 생성
    df_z_coeff = pd.DataFrame(coeff_results)
    return df_z_coeff, df_rawdata


if __name__ == "__main__":
    # 데이터 불러오기
    df_rawdata = pd.read_csv("RawData-1_2lot.csv")
    logging.info(f"Raw data loaded. Shape: {df_rawdata.shape}")

    # Zernike 분석 실행
    max_order = 5
    logging.info("Starting Zernike analysis")
    df_z_coeff, df_rawdata_with_predictions = zernike_analysis(df_rawdata, max_order=max_order)

    # 결과 저장
    df_z_coeff.to_csv("Zernike_Coefficients.csv", index=False)
    logging.info("Zernike coefficients saved to Zernike_Coefficients.csv")

    df_rawdata_with_predictions.to_csv("Z_FIT.csv", index=False)
    logging.info("Zernike predictions and residuals saved to Z_FIT.csv")


2024-11-23 04:52:25,935 - INFO - Raw data loaded. Shape: (2494, 38)
2024-11-23 04:52:25,936 - INFO - Starting Zernike analysis
2024-11-23 04:52:25,939 - INFO - Processing UNIQUE_ID: VH075030_PTVB827_3GJPBVH.VH075P_-_2024-05-30 15:39:37_2024-05-30 17:17:40_B3N049.1_11_E1
2024-11-23 04:52:25,947 - INFO - Processing UNIQUE_ID: WF075030_PTVP841_5G4PPWF.WF075P_-_2024-07-16 23:04:19_2024-07-16 23:55:04_PDS211.1_11_E1
2024-11-23 04:52:25,957 - INFO - Zernike coefficients saved to Zernike_Coefficients.csv
2024-11-23 04:52:26,019 - INFO - Zernike predictions and residuals saved to Z_FIT.csv
